# **Script pour obtenir les données**

*Ceci est un script de scraping des fichiers audio du site https://audio-lingua.ac-versailles.fr/ qui vont être utilisés pour le projet du cours de Réseaux de Neurones pour l'Oral dans le cadre du Master Traitement Automatique des Langues cohabilité par l'Université Paris Nanterre, Sorbonne Nouvelle et l'Inalco.*

Ce script prend plus de 5h d'éxecution. Les donnéees sont déjà dans le Drive donc pas besoin de le lancer. Nous voulions juste partager le au cas ou ! Bon courage.

In [2]:
### À installer pour pouvoir lancer le script ! ###

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 24.1 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.5 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Package

In [4]:
### Importer les librairies necessaires pour lancer les cellules qui suivent ! ###

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from requests.exceptions import RequestException
from typing import List
from dataclasses import dataclass
import csv
import argparse

In [5]:
### Definition des dataclass pour mieux gerer les informations extraites

@dataclass
class Audio:
    audio_language: str
    author: str
    title: str
    description: str
    date: str
    level: str
    gender: str
    age: str
    duration: str
    themes: List[str]
    file_name: str

@dataclass
class Language:
    audios: List[Audio]


In [6]:
### L'URL du site que nous allons scrapper

base_url = "https://audio-lingua.ac-versailles.fr/"

## **Definir les fonctions qui aideront à scraper le site.**

Ceci sont les fonctions qui permettent de parcourir la structure de site et de trouver les elements qui nous sont utiles pour telecharger les fichiers audio. Toutes ces fonctions sont lancées plus bas pour récupérer l'ensemble des données.

In [7]:
def get_base_page(base_url):

    """Fonction pour obtenir le contenu de la page de base
    Cette fonction envoie une requête HTTP à l'URL de base et utilise BeautifulSoup pour analyser le contenu HTML de la page d'accueil.
    Elle renvoie l'objet BeautifulSoup contenant les données de la page d'accueil."""

    homepage = requests.get(base_url)
    home_data = BeautifulSoup(homepage.content, 'lxml')
    return home_data

In [8]:
def choose_language(home_page, base_url):

    """Cette fonction sert a extraire les addresse de langue de la home page.
    On extrait les adresses des differentes langues disponibles sur la page d'accueil.
    Ensuite on parcourt les elements HTML contenant les liens vers les pages des langues et les ajoute a une liste."""

    language_adresses = []
    div_container = home_page.find_all("div", class_="fr-col-12 fr-col-sm-6 fr-col-lg-2")
    for child in div_container:
        if child:
            a_elem = child.find("a")
            link = a_elem.get("href")
            language_adress = base_url + link
            language_adresses.append(language_adress)
    return language_adresses

In [9]:
### On est obligés de cliquer sur des accordéons pour dérouler un petit menu qui contient des informations,
### et notamment les informations concernant les niveaux de langue.

def get_accordion(driver):

    """
    Focntion pour ouvrir tous les sections accordion de la page.
    En utilisant Selenium on interagit avec les elements de type accordion de la page.
    Grace a la librairie, on peut faire defiler la page et cliquer sur chaque bouton pour reveler le contenu cache.
    """

    accordion_buttons = driver.find_elements(By.CSS_SELECTOR, ".fr-accordion__btn")
    for button in accordion_buttons:
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(0.5)
        try:
            button.click()
        except:
            driver.execute_script("arguments[0].click();", button)
        time.sleep(1)
    return driver.page_source

In [10]:
def get_language_page(language_page):

    """Parser le contenu de la page qui contient les langues"""

    language_page_data = BeautifulSoup(language_page, "html.parser")
    return language_page_data

In [11]:
def extract_infos(language_page_data, args):

    """Extraire les informations sur l'audio comme l'auteur, la theme, l'age etc."""


    page_audios = []
    language = language_page_data.find("h1").text
    audios_sections = language_page_data.find_all("div", class_="fr-alert fr-alert--info mp3")
    for section in audios_sections:
        header = section.find("h3", class_="fr-alert__title")
        title = ''.join([str(content).strip() for content in header.contents if isinstance(content, str)])
        header_date_author = section.find("p", class_="fr-text--sm")
        for span in header_date_author.find_all("span"):
            if len(span["class"]) == 2:
                if "fr-icon-calendar-event-fill" in span["class"]:
                    date = (span.text).strip()
                elif "fr-icon-account-circle-fill" in span["class"]:
                    author = (span.text).strip()
        themes = []
        level, gender, age, duration = None, None, None, None
        tags = section.find("div", class_="fr-pt-2w")
        for tag in tags.find_all("a"):
            i = tag.find("i")
            class_i = i.get("class")
            if "icon-tag2" in class_i:
                level = (tag.text).strip()
            elif "icon-tag3" in class_i:
                gender = (tag.text).strip()
            elif "icon-tag4" in class_i:
                age = (tag.text).strip()
            elif "icon-tag5" in class_i:
                duration = (tag.text).strip()
            elif "icon-tag" in class_i:
                themes.append((tag.text).strip())
        quote = section.find("figure", class_="fr-quote")
        if quote:
            blockquote = quote.find("blockquote")
            if blockquote:
                description = (blockquote.text).strip()
        else:
            description = ""
        accordion = section.find("section")
        if accordion and level:
            download_section = accordion.find("p", class_="fr-text--sm fr-mt-2w")
            download_section = (download_section.text).split(":")
            download_link = download_section[1].strip() + ":" + download_section[2].strip()
            if " " in language:
                language = "_".join(language.split())
            if "/" in title:
                title = title.replace("/", "_")
            file_name = language + "_" + level + "_" + "_".join(title.split())
            if not download_audio(download_link, file_name, args):
                continue
        else:
            continue
        audio = Audio(audio_language=language, author=author, title=title, description=description, date=date,
                      level=level, gender=gender, age=age, duration=duration, themes=themes, file_name=file_name)
        page_audios.append(audio)
    return page_audios

In [12]:
def download_audio(download_link, file_name, language):

    """Telecharger un fichier audio a partir d'un lien"""

    max_retries = 5
    retry_delay = 5
    for attempt in range(max_retries):
        try:
            file_response = requests.get(download_link, timeout=10)
            file_response.raise_for_status()
            with open(f"../data/audios/{language}/{file_name}.mp3", "wb") as file:
                file.write(file_response.content)
            return True
        except RequestException as e:
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
            else:
                return False

In [13]:
def get_next_page_source(driver):

    """
    Cette fonction utilise Selenium pour cliquer sur le bouton de la page suivant et obtenir le contenu de la page.
    La fonction renvoie la source de la page suivante ou None si le bouton de la page suivante n'est pas disponible.
    """

    try: ### On met des try except car au vu du nombre de données, on peut avoir des temps de chargement trop lons !
        next_button = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Aller à la page suivante']")
        if not next_button.get_attribute("href"):
            return None
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(0.5)
        next_button.click()
        time.sleep(2)
        return driver.page_source
    except:
        return None

In [14]:
def build_csv(all_data, language):

    """Construire le fichier csv a partir des donnees extraites."""

    with open(f"../data/tables/{language}_test.csv", "w") as file:
        structure = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        structure.writerow(["Language", "FileName", "Author", "Title", "Description", "Date", "Level", "Gender", "Age", "Duration", "Themes"])
        for language in all_data:
            for audio in language.audios:
                structure.writerow([audio.audio_language, audio.file_name, audio.author, audio.title, audio.description, audio.date, audio.level, audio.gender, audio.age, audio.duration, audio.themes])

# **Le scrapping**

Cette partie coordonne l'exécution de toutes les autres fonctions pour extraire les données des audios et les enregistrer dans un fichier CSV.
Elle initialise le WebDriver, traite les pages de la langue française et construit le fichier CSV final.

In [15]:
    all_data = []

    homepage = get_base_page(base_url)
    language_adresses = choose_language(homepage, base_url)
    driver = webdriver.Chrome()

    language_adress = language_adresses[0]
    page = 1
    print(f"Processing Language : French...")
    print(f"Processing page {page}...")
    language = Language(audios=[])

    driver.get(language_adress)
    time.sleep(2)
    language_page = get_accordion(driver)
    language_page_data = get_language_page(language_page)

    audios = extract_infos(language_page_data, "Fr")
    language.audios.extend(audios)

    while True:
        next_page_source = get_next_page_source(driver)
        if not next_page_source:
            break
        page += 1
        print(f"Processing page {page}...")
        next_language_page = get_accordion(driver)
        next_language_page_data = get_language_page(next_language_page)
        language_page_data = next_language_page_data
        audios = extract_infos(language_page_data, "Fr")
        language.audios.extend(audios)
        if page == 10:
            break

    all_data.append(language)

    driver.quit()

    build_csv(all_data, "Fr")

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir
Stacktrace:
#0 0x55e155ca77ca <unknown>
#1 0x55e15579f2f0 <unknown>
#2 0x55e1557d6063 <unknown>
#3 0x55e1557d28c6 <unknown>
#4 0x55e15581fbc9 <unknown>
#5 0x55e15581f216 <unknown>
#6 0x55e155813753 <unknown>
#7 0x55e1557e0baa <unknown>
#8 0x55e1557e1dfe <unknown>
#9 0x55e155c7238b <unknown>
#10 0x55e155c76307 <unknown>
#11 0x55e155c5ee7c <unknown>
#12 0x55e155c76ec7 <unknown>
#13 0x55e155c4324f <unknown>
#14 0x55e155c962f8 <unknown>
#15 0x55e155c964c0 <unknown>
#16 0x55e155ca6646 <unknown>
#17 0x7ace8e25bac3 <unknown>
